import from google drive

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import Library yang Dibutuhkan

In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


 Muat Dataset

In [38]:
# Load dataset
file_path = '/content/drive/MyDrive/Folder rio/dataset/data_produk_fashion.csv'
data = pd.read_csv(file_path)

# Tampilkan beberapa baris pertama untuk memastikan data terbaca
data.head()


,item_name,description,category,price,colour,shop_name
0,Sepatu Sneakers,Sepatu Sneakers adalah pilihan yang sempurna u...,Sepatu,100000,Putih,Home Essentials
1,Tas Ransel Stylish,Tas Ransel Stylish memberikan gaya dan fungsi ...,Tas,200000,Merah,Sports World
2,Kemeja Flanel Modern,Kemeja Flanel Modern memberikan tampilan yang ...,Kemeja,150000,Hijau,Gourmet Delights
3,Celana Jeans Slim Fit,Celana Jeans Slim Fit adalah pilihan yang tepa...,Celana,250000,Biru,Gadget Hub
4,Topi Fedora Elegan,Topi Fedora Elegan memberikan sentuhan klasik ...,Topi,50000,Abu-abu,Gourmet Delights


 Preprocessing Teks pada Nama Produk

In [39]:
# Pastikan kolom sesuai dan hapus nilai yang kosong
data = data[['item_name', 'price']].dropna()

# Split data menjadi fitur (X) dan target (y)
X = data['item_name']
y = data['price']


Vektorisasi Nama Produk

In [40]:
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)


Split Data untuk Training dan Testing

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)


Latih Model Linear Regression

In [42]:
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

Evaluasi Model

In [43]:
# Prediksi pada data testing
y_pred = model.predict(X_test)

# Hitung MAE
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)


Mean Absolute Error (MAE): 63707.931997376516


Fungsi Prediksi untuk Input Nama Produk Baru

In [51]:
def adjust_price_based_on_condition(item_name, base_price):
    # Daftar kata kunci kondisi dan bobot diskon
    condition_keywords = {
    "baru": 1.0, "segel": 1.0, "original": 1.0, "authentic": 1.0,
    "bagus": 0.9, "baik": 0.9, "layak": 0.9, "bersih": 0.9, "terawat": 0.9,
    "terpakai": 0.8, "bekas": 0.8, "second": 0.8, "used": 0.8, "preloved": 0.8,
    "normal": 0.8, "sempurna": 0.8, "mulus": 0.8, "halus": 0.8,
    "kotor": 0.7, "berdebu": 0.7, "debu": 0.7, "tergores": 0.7, "warna pudar": 0.7,
    "usang": 0.7, "lama": 0.7, "belang": 0.7, "kusam": 0.7,
    "lecet": 0.6, "berminyak": 0.6, "terkelupas": 0.6, "retak kecil": 0.6, "penyok": 0.6,
    "terkikis": 0.6, "terbakar": 0.6, "menguning": 0.6, "berkerak": 0.6,
    "rusak": 0.5, "pecah": 0.5, "berlubang": 0.5, "sobek": 0.5, "retak": 0.5,
    "cacat": 0.5, "miring": 0.5, "tidak berfungsi": 0.5, "mati": 0.5,
    "parah": 0.4, "jelek": 0.4, "berjamur": 0.4, "berkarat": 0.4, "hancur": 0.4,
    "roboh": 0.4, "hilang bagian": 0.4, "tidak lengkap": 0.4, "tidak bisa dipakai": 0.4,
    "usang sekali": 0.3, "rusak parah": 0.3, "patah": 0.3, "jebol": 0.3, "hancur lebur": 0.3,
    "hangus": 0.3, "retak besar": 0.3, "luntur": 0.3, "hilang fungsi": 0.3,
    "diperbaiki": 0.6, "rekondisi": 0.6, "reparasi": 0.6, "restorasi": 0.6, "perawatan": 0.6,
    "retak halus": 0.7, "aus": 0.6, "berflek": 0.7, "warnanya memudar": 0.7,
    "pernah diperbaiki": 0.5, "bagian lepas": 0.5, "longgar": 0.7, "goyang": 0.7,
    "kendur": 0.7, "tidak kokoh": 0.6, "rapuh": 0.4, "mengeras": 0.6, "rapat": 0.9,
    "bergores": 0.7, "memudar": 0.6, "kusut": 0.6, "longgar": 0.6, "sobek halus": 0.6,
    "tambalan": 0.7, "plesteran": 0.7, "pernah dipakai": 0.8, "pakai sehari-hari": 0.9,
    "tidak sempurna": 0.5, "cacat pabrik": 0.7, "kain lusuh": 0.7, "serabut": 0.7,
    "tanda pemakaian": 0.7, "kulit retak": 0.5, "jaring berlubang": 0.5, "benang keluar": 0.5,
    "berlubang kecil": 0.6, "retak ringan": 0.7, "pengelupasan": 0.5, "bekas pakai": 0.8
}


  # Daftar kata kunci untuk gender
    gender_keywords = {
        "pria": 1.0,  # Misalnya, tidak ada pengaruh harga khusus untuk produk pria
        "wanita": 1.0  # Misalnya, produk wanita sedikit lebih mahal (10% lebih tinggi)
    }

    # Inisialisasi multiplier
    adjustment_multiplier = 1.0
    gender_multiplier = 1.0

    # Cek setiap kata kunci kondisi di nama produk
    for word, multiplier in condition_keywords.items():
        if word in item_name.lower():
            adjustment_multiplier = min(adjustment_multiplier, multiplier)  # Ambil diskon terbesar

    # Cek setiap kata kunci gender di nama produk
    for gender, gender_adjust in gender_keywords.items():
        if gender in item_name.lower():
            gender_multiplier = gender_adjust  # Gunakan multiplier berdasarkan gender

    # Hitung harga yang disesuaikan
    adjusted_price = base_price * adjustment_multiplier * gender_multiplier
    return adjusted_price

# Fungsi prediksi dengan penyesuaian
def predict_price_with_condition(item_name):
    base_price = predict_price(item_name)  # Prediksi harga dari model
    adjusted_price = adjust_price_based_on_condition(item_name, base_price)
    return adjusted_price

# Contoh penggunaan
nama_produk_baru = "blazer wanita kondisi masih baru"
print("Prediksi harga untuk '{}': Rp {:.2f}".format(nama_produk_baru, predict_price_with_condition(nama_produk_baru)))

Prediksi harga untuk 'blazer wanita kondisi masih baru': Rp 72721.57


In [54]:
import joblib

# Simpan model dan vectorizer
joblib.dump(model, '/content/drive/MyDrive/Folder rio/dataset/model_prediksi_harga.pkl')
joblib.dump(vectorizer, '/content/drive/MyDrive/Folder rio/dataset/vectorizer.pkl')


['/content/drive/MyDrive/Folder rio/dataset/vectorizer.pkl']